# Finetuning LLM for SAT Reading

## 1. Install and import necessary libaries

In [1]:
# !pip install -q -U bitsandbytes
# !pip install -q -U datasets
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U loralib
# !pip install -q -U einops

In [2]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

/home/tiviluson/miniconda3/envs/deeplearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load pre-trained LLM

In [ ]:
from transformers.utils.quantization_config import BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_NAME = "Qwen/Qwen3-4B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda:0",
    offload_state_dict=True,
    quantization_config=bnb_config,
)
print("Loading tokenizer...")

Loading tokenizer...


In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

In [11]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [12]:
model.print_trainable_parameters()

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


In [13]:
mem_free, mem_total = torch.cuda.mem_get_info()
print(f"Free memory: {mem_free / 1024**3:.2f} GB")
print(f"Total memory: {mem_total / 1024**3:.2f} GB")

Free memory: 12.01 GB
Total memory: 15.57 GB


In [14]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

## 3. Test pre-trained model performance

In [15]:
from transformers.generation.configuration_utils import GenerationConfig

In [16]:
# Llama-3's official system prompt structure
LLAMA3_SYSTEM_PROMPT = """You are a helpful AI assistant developed by Meta. Respond safely and accurately."""

In [17]:
# Test pretrained model performance
prompt = [
    {"role": "system", "content": LLAMA3_SYSTEM_PROMPT},
    {
        "role": "user",
        "content": """Analyze the given passage and question. Choose the best answer from the options below.

### Passage:
[This passage is adapted from George Eliot, Silas Marner. Originally published in 1861. Silas was a weaver and a notorious miser, but then the gold he had hoarded was stolen. Shortly after, Silas adopted a young child, Eppie, the daughter of an impoverished woman who had died suddenly.

    Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, with
trust in new joy, and stirring the human kindness in
all eyes that looked on her. The gold had kept his
thoughts in an ever-repeated circle, leading to
nothing beyond itself; but Eppie was an object
compacted of changes and hopes that forced his
thoughts onward, and carried them far away from
their old eager pacing towards the same blank
limit—carried them away to the new things that
would come with the coming years, when Eppie
would have learned to understand how her father
Silas cared for her; and made him look for images of
that time in the ties and charities that bound together
the families of his neighbors. The gold had asked that
he should sit weaving longer and longer, deafened
and blinded more and more to all things except the
monotony of his loom and the repetition of his web;
but Eppie called him away from his weaving, and
made him think all its pauses a holiday, reawakening
his senses with her fresh life, even to the old
winter-flies that came crawling forth in the early
spring sunshine, and warming him into joy because
she had joy.
    And when the sunshine grew strong and lasting,
so that the buttercups were thick in the meadows,
Silas might be seen in the sunny mid-day, or in the
late afternoon when the shadows were lengthening
under the hedgerows, strolling out with uncovered
head to carry Eppie beyond the Stone-pits to where
the flowers grew, till they reached some favorite bank
where he could sit down, while Eppie toddled to
pluck the flowers, and make remarks to the winged
things that murmured happily above the bright
petals, calling “Dad-dad’s” attention continually by
bringing him the flowers. Then she would turn her
ear to some sudden bird-note, and Silas learned to
please her by making signs of hushed stillness, that
they might listen for the note to come again: so that
when it came, she set up her small back and laughed
with gurgling triumph. Sitting on the banks in this
way, Silas began to look for the once familiar herbs
again; and as the leaves, with their unchanged outline
and markings, lay on his palm, there was a sense of
crowding remembrances from which he turned away
timidly, taking refuge in Eppie’s little world, that lay
lightly on his enfeebled spirit.
    As the child’s mind was growing into knowledge,
his mind was growing into memory: as her life
unfolded, his soul, long stupefied in a cold narrow
prison, was unfolding too, and trembling gradually
into full consciousness.
    It was an influence which must gather force with
every new year: the tones that stirred Silas’ heart
grew articulate, and called for more distinct answers;
shapes and sounds grew clearer for Eppie’s eyes and
ears, and there was more that “Dad-dad” was
imperatively required to notice and account for.
Also, by the time Eppie was three years old, she
developed a fine capacity for mischief, and for
devising ingenious ways of being troublesome, which
found much exercise, not only for Silas’ patience, but
for his watchfulness and penetration. Sorely was poor
Silas puzzled on such occasions by the incompatible
demands of love.]

### Question:
Which statement best describes a technique the narrator uses to represent Silas's character before he adopted Eppie?

### Choices:
A) The narrator emphasizes Silas's former obsession with wealth by depicting his gold as requiring certain behaviors on his part.
B) The narrator underscores Silas's former greed by describing his gold as seeming to reproduce on its own.
C) The narrator hints at Silas's former antisocial attitude by contrasting his present behavior toward his neighbors with his past behavior toward them.
D) The narrator demonstrates Silas's former lack of self-awareness by implying that he is unable to recall life before Eppie.

Respond ONLY with the letter and full text of the correct answer choice.""",
    },
]

In [18]:
# Apply chat template (text only)
chat_text = tokenizer.apply_chat_template(
    prompt, add_generation_prompt=True, tokenize=False
)
print(chat_text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 May 2025

You are a helpful AI assistant developed by Meta. Respond safely and accurately.<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze the given passage and question. Choose the best answer from the options below.

### Passage:
[This passage is adapted from George Eliot, Silas Marner. Originally published in 1861. Silas was a weaver and a notorious miser, but then the gold he had hoarded was stolen. Shortly after, Silas adopted a young child, Eppie, the daughter of an impoverished woman who had died suddenly.

    Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, 

In [19]:
tokenizer("<|begin_of_text|><|end_of_text|>")

{'input_ids': [128000, 128000, 128001], 'attention_mask': [1, 1, 1]}

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenize
inputs = tokenizer(chat_text, return_tensors="pt").to(device)

for k, v in inputs.items():
    print(f"{k}: {v.shape}")
v.all()

input_ids: torch.Size([1, 1104])
attention_mask: torch.Size([1, 1104])


tensor(True, device='cuda:0')

In [21]:
generation_config = GenerationConfig(
    max_new_tokens=50,
    temperature=0.01,
    do_sample=True,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.3,
)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        generation_config=generation_config,
    )

    # Decode output
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


# print
if "<|assistant|>" in output_text:
    print(output_text.split("<|assistant|>")[-1].strip())
else:
    print(output_text.strip())

system

Cutting Knowledge Date: December 2023
Today Date: 04 May 2025

You are a helpful AI assistant developed by Meta. Respond safely and accurately.user

Analyze the given passage and question. Choose the best answer from the options below.

### Passage:
[This passage is adapted from George Eliot, Silas Marner. Originally published in 1861. Silas was a weaver and a notorious miser, but then the gold he had hoarded was stolen. Shortly after, Silas adopted a young child, Eppie, the daughter of an impoverished woman who had died suddenly.

    Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, with
trust in new joy, and stirring the human kindness in
all eyes that looked on her. The gold had

## 4. Fine-tuning LLM

### 4.1. Prepare dataset

In [22]:
from datasets.dataset_dict import DatasetDict

In [23]:
data: DatasetDict = load_dataset("emozilla/sat-reading")  # type: ignore

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 298
    })
    validation: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 39
    })
    test: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 38
    })
})

In [25]:
type(data)

datasets.dataset_dict.DatasetDict

In [26]:
data["train"]

Dataset({
    features: ['text', 'answer', 'requires_line', 'id'],
    num_rows: 298
})

In [27]:
print(data["train"]["text"][4])

SAT READING COMPREHENSION TEST

This passage is adapted from George Eliot, Silas Marner.
Originally published in 1861. Silas was a weaver and a
notorious miser, but then the gold he had hoarded was
stolen. Shortly after, Silas adopted a young child, Eppie, the
daughter of an impoverished woman who had died
suddenly.

    Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, with
trust in new joy, and stirring the human kindness in
all eyes that looked on her. The gold had kept his
thoughts in an ever-repeated circle, leading to
nothing beyond itself; but Eppie was an object
compacted of changes and hopes that forced his
thoughts onward, and carried them far away from
their old eager pacing towa

In [28]:
print(data["train"]["answer"][4])

B


In [29]:
import re


def extract_sections(text):
    """Parse raw SAT text into structured sections"""
    sections = {"passage": "", "question": "", "choices": [], "answer_letter": ""}

    # Tách phần answer
    match = re.search(r"Answer:\s*([A-E])", text, re.IGNORECASE)
    sections["answer_letter"] = match.group(1) if match else ""

    # Tách các phần chính
    # Greedy matching with *
    match = re.search(
        r"SAT READING COMPREHENSION TEST(.*)Answer:", text, re.DOTALL | re.IGNORECASE
    )
    content = match.group(1).strip() if match else ""
    blocks = [b.strip() for b in re.split(r"\n\n", content) if b.strip()]
    sections["passage"] = "\n".join(blocks[0:2])
    question_block = blocks[2].splitlines()
    sections["question"] = question_block[1]
    sections["choices"] = question_block[2:]

    return sections

In [30]:
extract_sections(data["train"]["text"][4])

{'passage': 'This passage is adapted from George Eliot, Silas Marner.\nOriginally published in 1861. Silas was a weaver and a\nnotorious miser, but then the gold he had hoarded was\nstolen. Shortly after, Silas adopted a young child, Eppie, the\ndaughter of an impoverished woman who had died\nsuddenly.\nUnlike the gold which needed nothing, and must\nbe worshipped in close-locked solitude—which was\nhidden away from the daylight, was deaf to the song\nof birds, and started to no human tones—Eppie was a\ncreature of endless claims and ever-growing desires,\nseeking and loving sunshine, and living sounds, and\nliving movements; making trial of everything, with\ntrust in new joy, and stirring the human kindness in\nall eyes that looked on her. The gold had kept his\nthoughts in an ever-repeated circle, leading to\nnothing beyond itself; but Eppie was an object\ncompacted of changes and hopes that forced his\nthoughts onward, and carried them far away from\ntheir old eager pacing towards t

In [31]:
def map_answer(text, letter):
    """Match answer letter with full choice text"""
    sections = extract_sections(text)
    for choice in sections["choices"]:
        if choice.startswith(f"{letter})"):
            return choice
    return letter  # Fallback if not found

In [32]:
def generate_prompt(text, answer_letter):
    sections = extract_sections(text)

    choices_text = "\n".join(sections["choices"])

    return [
        {"role": "system", "content": LLAMA3_SYSTEM_PROMPT},
        {
            "role": "user",
            "content": f"""Read the passage and answer the question.

### Passage:
{sections['passage']}

### Question:
{sections['question']}

### Choices:
{choices_text}

Respond with ONLY the letter and full text of the correct answer.""",
        },
        {"role": "assistant", "content": map_answer(text, answer_letter)},
    ]

In [33]:
def generate_and_tokenize_prompt(user_input, answer):
    try:
        full_prompt = generate_prompt(user_input, answer)

        prompt_str = tokenizer.apply_chat_template(
            full_prompt, tokenize=False, add_generation_prompt=False
        )
        tokenized = tokenizer(
            prompt_str,
            # padding="max_length",
            # truncation=True,
            # max_length=1500,
            return_tensors="pt",
        )

        input_ids = tokenized["input_ids"][0]
        # labels = input_ids.clone()

        return {
            "input_ids": input_ids,
            "attention_mask": tokenized["attention_mask"][0],
            # "labels": labels,
        }

    except Exception as e:
        print(f"Error processing sample: {e}")
        return None

In [34]:
sample_text = data["train"]["text"][0]
sample_answer = data["train"]["answer"][0]

example_messages = generate_prompt(sample_text, sample_answer)
prompt_text = tokenizer.apply_chat_template(
    example_messages, tokenize=False, add_generation_prompt=False
)
print("=== Generated Prompt ===")
print(prompt_text)

=== Generated Prompt ===
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 May 2025

You are a helpful AI assistant developed by Meta. Respond safely and accurately.<|eot_id|><|start_header_id|>user<|end_header_id|>

Read the passage and answer the question.

### Passage:
This passage is adapted from George Eliot, Silas Marner.
Originally published in 1861. Silas was a weaver and a
notorious miser, but then the gold he had hoarded was
stolen. Shortly after, Silas adopted a young child, Eppie, the
daughter of an impoverished woman who had died
suddenly.
Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, with
trust in new joy, and

In [35]:
tokenized_sample = generate_and_tokenize_prompt(sample_text, sample_answer)
if tokenized_sample:
    print("\n=== Tokenized Sample ===")
    print(f"Input IDs shape: {tokenized_sample['input_ids'].shape}")
    print(f"Sample decoded back:")
    print(tokenizer.decode(tokenized_sample["input_ids"], skip_special_tokens=True))


=== Tokenized Sample ===
Input IDs shape: torch.Size([1124])
Sample decoded back:
system

Cutting Knowledge Date: December 2023
Today Date: 04 May 2025

You are a helpful AI assistant developed by Meta. Respond safely and accurately.user

Read the passage and answer the question.

### Passage:
This passage is adapted from George Eliot, Silas Marner.
Originally published in 1861. Silas was a weaver and a
notorious miser, but then the gold he had hoarded was
stolen. Shortly after, Silas adopted a young child, Eppie, the
daughter of an impoverished woman who had died
suddenly.
Unlike the gold which needed nothing, and must
be worshipped in close-locked solitude—which was
hidden away from the daylight, was deaf to the song
of birds, and started to no human tones—Eppie was a
creature of endless claims and ever-growing desires,
seeking and loving sunshine, and living sounds, and
living movements; making trial of everything, with
trust in new joy, and stirring the human kindness in
all eyes 

In [36]:
def preprocess_dataset(sample):
    tokenized_sample = None
    try:
        processed_text = sample["text"]
        processed_answer = map_answer(sample["text"], sample["answer"].strip())
        tokenized_sample = generate_and_tokenize_prompt(
            processed_text, processed_answer
        )
    except Exception as e:
        print(f"Skipping invalid sample: {e}")
    return tokenized_sample

In [37]:
dataset = data.map(
    preprocess_dataset,
    remove_columns=data["train"].column_names,
    num_proc=4,
    desc="Processing training dataset",
).filter(
    lambda x: x is not None,
    desc="Filtering out invalid samples",
)

In [38]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 298
    })
    validation: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 39
    })
    test: Dataset({
        features: ['text', 'answer', 'requires_line', 'id'],
        num_rows: 38
    })
})

In [39]:
# check length samples
train_lengths = [len(x["input_ids"]) for x in dataset["train"]]  # type: ignore
eval_lengths = [len(x["input_ids"]) for x in dataset["validation"]]  # type: ignore
all_lengths = train_lengths + eval_lengths
print(
    f"Min: {min(all_lengths)}, Max: {max(all_lengths)}, Avg: {sum(all_lengths)/len(all_lengths):.1f}"
)

Min: 864, Max: 1396, Avg: 1128.0


### 4.2. Training

In [40]:
from transformers.trainer_callback import TrainerCallback
from rich.console import Console
from rich.table import Table


class LogLossCallback(TrainerCallback):
    def __init__(self):
        self.console = Console()
        self.table = Table(show_header=True, header_style="bold magenta")
        self.table.add_column("Step", justify="right")
        self.table.add_column("Training Loss", justify="right")
        self.logged_steps = set()

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            step = state.global_step
            if step not in self.logged_steps:
                loss = logs["loss"]
                self.table.add_row(str(step), f"{loss:.6f}")
                self.logged_steps.add(step)

                # if step % 10 == 0:
                # self.console.clear()
                self.console.print(self.table)

In [41]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
  

In [42]:
from peft import PeftModelForCausalLM

PeftModelForCausalLM.__mro__

(peft.peft_model.PeftModelForCausalLM,
 peft.peft_model.PeftModel,
 transformers.utils.hub.PushToHubMixin,
 torch.nn.modules.module.Module,
 object)

In [43]:
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer
from transformers.data.data_collator import DataCollatorForLanguageModeling

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir="llama3-8b-sat-reading",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    report_to="none",
    remove_unused_columns=False,
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8
)

In [44]:
# dataset = dataset.remove_columns(["labels"])

In [45]:
# Quantization-aware training settings
model.config.use_cache = False
model.enable_input_require_grads()
# compiled_model = torch.compile(model)


trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    args=training_args,
    data_collator=data_collator,
    callbacks=[LogLossCallback()],
)


trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tiviluson/miniconda3/envs/deeplearning/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


┏━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Step ┃ Training Loss ┃
┡━━━━━━╇━━━━━━━━━━━━━━━┩
│   10 │      2.904600 │
└──────┴───────────────┘

┏━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Step ┃ Training Loss ┃
┡━━━━━━╇━━━━━━━━━━━━━━━┩
│   10 │      2.904600 │
│   20 │      2.637800 │
└──────┴───────────────┘

┏━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Step ┃ Training Loss ┃
┡━━━━━━╇━━━━━━━━━━━━━━━┩
│   10 │      2.904600 │
│   20 │      2.637800 │
│   30 │      2.334800 │
└──────┴───────────────┘

┏━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Step ┃ Training Loss ┃
┡━━━━━━╇━━━━━━━━━━━━━━━┩
│   10 │      2.904600 │
│   20 │      2.637800 │
│   30 │      2.334800 │
│   40 │      2.335100 │
└──────┴───────────────┘

┏━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Step ┃ Training Loss ┃
┡━━━━━━╇━━━━━━━━━━━━━━━┩
│   10 │      2.904600 │
│   20 │      2.637800 │
│   30 │      2.334800 │
│   40 │      2.335100 │
│   50 │      2.202500 │
└──────┴───────────────┘

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.68 GiB. GPU 